<a href="https://colab.research.google.com/github/Bertha-ding/independent-study/blob/main/phase3_revised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize as so
import scipy.stats as ss

'''=========
option class init
=========='''
class VanillaOption:
    def __init__(
        self,
        otype = 1, # 1: 'call'
                  # -1: 'put'
        strike = 110.,
        maturity = 1.,
        market_price = 10.):
      self.otype = otype
      self.strike = strike
      self.maturity = maturity
      self.market_price = market_price #this will be used for calibration
      
        
    def payoff(self, s): #s: excercise price
      otype = self.otype
      k = self.strike
      maturity = self.maturity
      return max([0, (s - k)*otype])
'''============
Gbm class
============='''

class Gbm:
    def __init__(self,
                 init_state = 100.,
                 drift_ratio = .0475,
                 vol_ratio = .2,
                ):
        self.init_state = init_state
        self.drift_ratio = drift_ratio
        self.vol_ratio = vol_ratio


'''========
Black-Scholes-Merton formula. 
=========='''

def bsm_price(self, vanilla_option):
    s0 = self.init_state
    sigma = self.vol_ratio
    r = self.drift_ratio
    
    otype = vanilla_option.otype
    k = vanilla_option.strike
    maturity = vanilla_option.maturity
    
    d1 = (np.log(s0 / k) + (r + 0.5 * sigma ** 2) 
          * maturity) / (sigma * np.sqrt(maturity))
    d2 = d1 - sigma * np.sqrt(maturity)
    
    return (otype * s0 * ss.norm.cdf(otype * d1) #line break needs parenthesis
            - otype * np.exp(-r * maturity) * k * ss.norm.cdf(otype * d2))

Gbm.bsm_price = bsm_price


In [23]:
%cd~

!git clone https://github.com/Bertha-ding/20MA573-yuning-ding.git 
pass

%cd 20MA573-yuning-ding/src/
%ls

/root
fatal: destination path '20MA573-yuning-ding' already exists and is not an empty directory.
/root/20MA573-yuning-ding/src
20optiondata2.dat  hw1_random_walk.ipynb  prj02.ipynb  __pycache__/
bsm.py             prj01.ipynb            prj03.ipynb  Untitled0.ipynb


In [24]:
from bsm import *
import scipy.optimize as so
import numpy as np
import scipy.stats as ss

#Read four-column data
#columns are otype, maturity, strike, option_price
np_option_data1 = np.loadtxt('20optiondata2.dat', comments='#', delimiter=',')

print('>>>otype,>>>>>>> maturity, >>strike, >>>>>>option_price')
print(np_option_data1)

>>>otype,>>>>>>> maturity, >>strike, >>>>>>option_price
[[  1.           0.16666667  97.           5.32705461]
 [  1.           0.16666667  99.           3.86224255]
 [  1.           0.16666667 101.           2.7204371 ]
 [  1.           0.16666667 103.           2.1202793 ]
 [  1.           0.41666667  97.           7.23756307]
 [  1.           0.41666667  99.           5.95053461]
 [  1.           0.41666667 101.           5.2640122 ]
 [  1.           0.41666667 103.           4.97493422]]


In [28]:
gbm1 = Gbm(
    init_state = 100., 
    drift_ratio = .04,
    vol_ratio = .2)

In [5]:
'''================
define an error function
===================='''
def error_function(vol, gbm, option_list):
  gbm.vol_ratio = vol
  err = 0
  for i in range(len(option_list)):
    err = (err + abs((option_list[i]).market_price - gbm.bsm_price(option_list[i]))**2)/len(option_list)   
  return err

  


In [101]:
def dx1(vol,init_state,drift_ratio, gbm, option_list):
  gbm.vol_ratio = vol
  gbm.init_state = init_state
  gbm.drift_ratio= drift_ratio
  for i in range(len(option_list)):
    gradient= abs((option_list[i]).market_price - gbm.bsm_price(option_list[i]))/len(option_list) 
    gradient=(gbm.bsm_price(option_list[i])-(option_list[i]).market_price)*100*(np.log(100/90) + (0.04 + 0.5 *vol ** 2) 
          * 2/12) / (vol * np.sqrt(2/12))*np.sqrt(2/12)
  return gradient

def gradient(vol,init_state,drift_ratio, gbm, option_list):
  #gradient_w1:estimate mean gradient over all point for w1
  #1/N*sum of derivative of w1
  N=len(option_list)
  total=0
  for i in range(len(option_list)):
    total +=dx1(vol,init_state,drift_ratio, gbm, option_list)
  gradient = 2*total/N
  return gradient


In [106]:
def gradient_descent(vol,init_state,drift_ratio,gbm,option_list,alpha,precision):
  vol_list = []
  while abs(vol - 0.2) > precision:
      grad = gradient(vol,init_state,drift_ratio,gbm,option_list)
      vol = vol - alpha*grad
      vol_list.append(vol_list)
  print("Local minimum occurs at:", vol)
  print("Number of steps:", len(vol_list))

In [114]:
def implied_volatility(gbm, init_state,drift_ratio,option_list):
  init_vol = .3 #initial guess
  return gradient_descent(init_vol,init_state,drift_ratio,gbm,option_list,alpha=0.001,precision=0.001)

In [112]:
filter1 = np_option_data1[np_option_data1[:,1] == 2/12]
num_row = filter1.shape[0]
option_list = []

for i in range(num_row):
  option1 = VanillaOption(
      otype = filter1[i,0],
      strike = filter1[i,2],
      maturity = filter1[i,1],
      market_price = filter1[i,3]
  )   
  option_list.append(option1)

In [115]:
implied_volatility(gbm1, 100,0.04,option_list)

Local minimum occurs at: 0.20023401019107984
Number of steps: 12
